# Introducción al Clustering
El clustering es una técnica de aprendizaje no supervisado que consiste en agrupar objetos similares en grupos o clusters. El objetivo principal de esta técnica es encontrar una estructura subyacente en un conjunto de datos y categorizarlos en grupos o clases basados en sus características. Algunas aplicaciones comunes de clustering incluyen segmentación de clientes, análisis de redes sociales, detección de anomalías y agrupamiento de documentos.

Existen varios algoritmos de clustering, como K-means, DBSCAN, Clustering Jerárquico y Clustering Espectral. En este cuaderno nos enfocaremos en el algoritmo K-means.

## Algoritmo K-means
K-means es un algoritmo de clustering basado en particiones. Se trata de un algoritmo iterativo que intenta minimizar la suma de las distancias al cuadrado entre los puntos y el centroide de su respectivo cluster (inercia o varianza dentro del grupo). El algoritmo se ejecuta de la siguiente manera:

1. Elegir el número de clusters, `k`.
2. Asignar de manera aleatoria `k` puntos como centroides iniciales.
3. Asignar cada punto al centroide más cercano.
4. Actualizar los centroides calculando el promedio de todos los puntos asignados a cada centroide.
5. Repetir los pasos 3 y 4 hasta que los centroides no cambien significativamente o se alcance un número máximo de iteraciones.

La elección del número de clusters, `k`, es un aspecto crítico del algoritmo K-means. Un valor de `k` demasiado pequeño puede resultar en un agrupamiento deficiente, mientras que un valor demasiado grande puede resultar en un agrupamiento excesivamente detallado. Se pueden utilizar diversas técnicas para determinar el valor óptimo de `k`, como el método del codo o la silueta.

A continuación, vamos a generar un conjunto de datos sintéticos y aplicar el algoritmo K-means utilizando la biblioteca `sklearn`. Después, visualizaremos los resultados con `plotly`.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')

# Crear un conjunto de datos sintéticos
X, y_true = make_blobs(n_samples=1000,
                        centers=4,
                        cluster_std=0.2,
                        random_state=0)

In [ ]:
# Aplicar el algoritmo K-means
k = 4
kmeans = KMeans(n_clusters=k,
                init='k-means++',
                max_iter=300,
                n_init=10,
                random_state=0)
y_kmeans = kmeans.fit_predict(X)


In [ ]:
# Visualizar los resultados con plotly
df = pd.DataFrame(X, columns=['x', 'y'])
df['cluster'] = y_kmeans
df['cluster'] = y_kmeans.astype(str)

fig = px.scatter(df, x='x', y='y', color='cluster',
                 title='Resultado de K-means (k=4)',
                 color_discrete_sequence=px.colors.qualitative.D3)

# Añadir los centroides
fig.add_scatter(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1],
                mode='markers', marker=dict(symbol='x', size=12, color='black'),
                name='Centroides')

# Ajustes del layout para mantener la misma escala y aumentar el tamaño
fig.update_layout(
    width=800,  # Ajusta la anchura de la figura
    height=600,  # Ajusta la altura de la figura
    xaxis=dict(scaleanchor="y", scaleratio=1),
    title=dict(x=0.5)  # Centra el título
)

fig.show()

El código anterior crea un conjunto de datos sintéticos con 300 puntos y 4 clusters. Luego, aplicamos el algoritmo K-means utilizando `sklearn.cluster.KMeans`. Utilizamos el método 'k-means++' para inicializar los centroides, lo que ayuda a acelerar la convergencia y mejorar la calidad de la solución. Ejecutamos el algoritmo durante un máximo de 300 iteraciones y lo inicializamos 10 veces con diferentes centroides iniciales. Luego, visualizamos los resultados con `plotly`.

En este ejemplo, el algoritmo K-means agrupó los puntos en 4 clusters, como se esperaba. Sin embargo, en casos reales, donde no conocemos la cantidad óptima de clusters, podemos utilizar algún método para determinar el valor óptimo de `k`.

## Método del codo

El método del codo es una técnica heurística para determinar el número óptimo de clusters (`k`) en un conjunto de datos cuando se utiliza el algoritmo K-means. La idea principal es ejecutar K-means con diferentes valores de `k` y calcular la inercia (también conocida como suma de las distancias al cuadrado entre los puntos y el centroide de su respectivo cluster) para cada valor de `k`. Luego, se grafica la inercia en función de `k`, y el punto en el que la curva comienza a disminuir más lentamente (similar a un "codo") se considera el valor óptimo de `k`. La inercia es una medida de la cohesión de los clusters, es decir, qué tan cerca están los puntos dentro de un cluster. Un valor de inercia más bajo indica que los puntos dentro de un cluster están más cerca entre sí.

In [ ]:
inertias = []
ks = range(1,11) #[1,2,3,4,5,6,7,8,9,10]
for k in ks:
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)

# Visualizar el método del codo
fig = px.line(x=ks, y=inertias, title='Método del codo',
              labels={'x': 'Número de clusters (k)', 'y': 'Inercia'})
fig.show()

En el código anterior, calculamos la inercia para diferentes valores de `k` (desde 1 hasta 10) y visualizamos los resultados utilizando el método del codo. El punto en el que la curva de inercia comienza a disminuir más lentamente (similar a un "codo") se considera el valor óptimo de `k`. En este caso, el método del codo sugiere que `k=4` es un buen valor para nuestro conjunto de datos sintético.

En pocas palabras, el algoritmo K-means es una técnica de clustering útil y fácil de implementar que puede utilizarse en diversas aplicaciones de ciencia de datos. Es importante tener en cuenta que este algoritmo es sensible a la inicialización de los centroides y al número de clusters, por lo que es recomendable utilizar métodos como 'k-means++' para la inicialización y técnicas como el método del codo para determinar el valor óptimo de `k`. También es importante mencionar que el algoritmo K-means asume que los clusters son esféricos y de tamaño similar, lo cual puede no ser siempre cierto en problemas del mundo real. En tales casos, otros algoritmos de clustering, como DBSCAN o Clustering Jerárquico, podrían ser más apropiados.

Además, es fundamental tener en cuenta que el algoritmo K-means es sensible a la escala de las características. Por lo tanto, es recomendable estandarizar o normalizar los datos antes de aplicar el algoritmo, especialmente cuando las características tienen diferentes unidades de medida o rangos de valores.

Un ejemplo de cómo normalizar los datos utilizando `StandardScaler` de `sklearn` se presenta a continuación:

In [ ]:
from sklearn.preprocessing import StandardScaler

# Crear datos de ejemplo
data = np.array([[1, 2000], [2, 3000], [3, 4000], [4, 5000]])

# Estandarizar los datos
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

print("Datos estandarizados:")
print(data_scaled)

`StandardScaler` es una clase de la biblioteca `sklearn` que se utiliza para estandarizar las características de un conjunto de datos. La estandarización es un proceso de preprocesamiento de datos que transforma las características para que tengan una media de 0 y una desviación estándar de 1. Esto es útil en muchos algoritmos de aprendizaje automático, como K-means y otros algoritmos de clustering, ya que estos algoritmos son sensibles a la escala de las características.

La fórmula de la estandarización para cada característica es:

$$
z = \frac{x - \mu}{\sigma}
$$

donde:

* $z$: valor estandarizado
* $x$: valor original de la característica
* $\mu$: media de la característica en el conjunto de datos
* $\sigma$: desviación estándar de la característica en el conjunto de datos

En el ejemplo anterior, primero creamos un conjunto de datos de ejemplo con dos características. Luego, inicializamos un objeto `StandardScaler` y lo ajustamos a los datos de ejemplo utilizando el método `fit_transform`. Esto calcula la media y la desviación estándar de cada característica en el conjunto de datos y aplica la estandarización a los datos de ejemplo. El resultado son los datos estandarizados, donde cada característica tiene una media de 0 y una desviación estándar de 1.

Al aplicar K-means a datos estandarizados, el algoritmo puede ser más efectivo en la identificación de clusters con formas y tamaños diferentes.

## Ejemplo

A continuación, se presenta un ejemplo de cómo utilizar el algoritmo K-means en un conjunto de datos real: el conjunto de datos de Wine. Este conjunto de datos contiene las mediciones químicas de 178 muestras de vino producidas en Italia. Estas muestras pertenecen a tres diferentes cultivares. Las características medidas incluyen, entre otras, el alcohol, la intensidad del color y el nivel de proline, lo cual es crucial para determinar la calidad y las características del vino. Aunque las etiquetas de los cultivares están disponibles en este conjunto de datos, no las utilizaremos para el algoritmo K-means, ya que es un algoritmo de aprendizaje no supervisado.

Este análisis ayudará a ilustrar cómo la estandarización de las características puede impactar los resultados del algoritmo K-means, especialmente cuando las características tienen diferentes escalas. Utilizaremos las características de alcohol, intensidad del color y proline, que varían ampliamente en magnitud y son significativas para diferenciar entre diferentes tipos de vino. La visualización en 3D de los clusters permitirá apreciar mejor cómo la estandarización influye en la agrupación de los datos.

In [ ]:
from sklearn.datasets import load_wine
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Cargar el conjunto de datos de Wine
data = load_wine()
X = data.data
features = data.feature_names

# Seleccionar tres características importantes
important_features = ['alcohol', 'color_intensity', 'proline']
indices = [features.index(feature) for feature in important_features]
X_selected = X[:, indices]

# Datos no escalados
kmeans_original = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
y_kmeans_original = kmeans_original.fit_predict(X_selected)
df_original = pd.DataFrame(X_selected, columns=important_features)
df_original['cluster'] = y_kmeans_original.astype(str)  # Convertir a string para tratar como categórico

# Estandarizar las características seleccionadas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
kmeans_scaled = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
y_kmeans_scaled = kmeans_scaled.fit_predict(X_scaled)
df_scaled = pd.DataFrame(X_scaled, columns=important_features)
df_scaled['cluster'] = y_kmeans_scaled.astype(str)  # Convertir a string para tratar como categórico

In [ ]:
# Crear subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('K-means en datos originales', 'K-means en datos escalados'),
                    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

# Paleta de colores categóricos
colors = px.colors.qualitative.Plotly

# Añadir gráficas de datos originales
fig.add_trace(go.Scatter3d(x=df_original['alcohol'], y=df_original['color_intensity'], z=df_original['proline'],
                           mode='markers', marker=dict(size=5, color=[colors[int(i)] for i in df_original['cluster']]),
                           name='Clusters originales'), row=1, col=1)

# Añadir gráficas de datos escalados
fig.add_trace(go.Scatter3d(x=df_scaled['alcohol'], y=df_scaled['color_intensity'], z=df_scaled['proline'],
                           mode='markers', marker=dict(size=5, color=[colors[int(i)] for i in df_scaled['cluster']]),
                           name='Clusters escalados'), row=1, col=2)

# Actualizar el layout para mejorar la visualización
fig.update_layout(height=600, width=1200, showlegend=True)

fig.show()

En este ejemplo, después de cargar y seleccionar las características relevantes del conjunto de datos de Wine, aplicamos el algoritmo K-means con `k=3`, correspondiente al número de tipos de vinos en el conjunto. Utilizamos un gráfico 3D de Plotly para visualizar los resultados, donde cada punto representa una muestra de vino y los colores indican los clusters formados por K-means. Los centroides de los clusters están destacados como cruces negras.

Es interesante observar cómo la estandarización de las características influye significativamente en la agrupación. Específicamente, la variable "proline", que típicamente tiene valores en el rango de miles, domina la medida de distancia en el algoritmo K-means cuando los datos no están estandarizados. Esto puede llevar a una agrupación ineficaz, ya que las diferencias en las escalas de las variables sesgan los resultados hacia las características de mayor magnitud. Al estandarizar los datos, cada característica contribuye equitativamente al análisis, permitiendo que K-means identifique patrones más sutiles y efectivos entre las muestras de vino.

Este comportamiento subraya la utilidad de K-means para explorar y descubrir estructuras ocultas en conjuntos de datos sin etiquetas, demostrando la importancia de la preparación adecuada de los datos antes de aplicar técnicas de aprendizaje automático.